# Parameter Tuning 
* LGBM 
* Bayesian Optimization 
* GPU ver.

In [1]:
import os
import sys
import joblib
import pickle as pkl

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore')
warnings.filterwarnings(action='ignore', category=DataConversionWarning)


import pandas as pd 
import numpy as np


from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

In [8]:
cd /content/drive/My Drive/쇼핑광고등어/3. Modeling/재빈

/content/drive/My Drive/쇼핑광고등어/3. Modeling/재빈


In [9]:
import BoostingModel as model
import SHAPvalue as SHAP
import optimization as opt

In [10]:
data_v4 = joblib.load(os.path.join('..', '..', '1. Data', '05_분석데이터', '7th_train_FE.pkl'))
locals().update(data_v4)

In [11]:
X = data_v4['X'] ; y = data_v4['y']
X.shape, y.shape

((35379, 402), (35379,))

In [15]:
y2 = np.log1p(y)

## Modeling

### lgbm_params1

In [19]:
from bayes_opt import BayesianOptimization

In [13]:
params_range = {
                'num_leaves': (8, 64),
                'max_depth': (4, 64),
                'min_child_samples': (16, 64),  
                'reg_lambda': (0.01, 0.99), 
                'min_split_gain': (0.005, 0.035),
                'colsample_bytree': (0.5, 0.9), 
                'subsample': (0.7, 0.9), 
                'subsample_freq': (0, 7),
                'max_bin': (30, 75), 
                'learning_rate' : (.001, .01)
               }

In [23]:
def Bayes_running(num_leaves, max_depth, min_child_samples, reg_lambda, min_split_gain, colsample_bytree, subsample, subsample_freq, max_bin, learning_rate):
    params = { 
                'boosting': 'gbdt', 'objective': 'regression', 'metric': 'mape', 'is_training_metric': True, 'n_estimators' : 25000, 
              
                "num_leaves": int(round(num_leaves)),
                "max_depth" : int(round(max_depth)),
                'min_child_samples': int(round(min_child_samples)), 
                'reg_lambda': max(reg_lambda, 0), 
                "min_split_gain": min_split_gain,
                'colsample_bytree': colsample_bytree, 
                'subsample': max(min(subsample,1),0), 
                'subsample_freq': int(round(subsample_freq)),
                'max_bin': int(round(max_bin)), 
              
                "learning_rate": learning_rate, 
                "n_jobs" : -1,
                'device_type' : 'gpu', 
                'verbose' : -1
              
    }
    
    print(params)

    mape, pred = opt.lgbm_model(X, y2, params, cv_splits=5, scaling='MinMax', epoch=25000)
    return -(mape['final_mape'][0])

In [24]:
%%time 

Bayesian = BayesianOptimization(Bayes_running, params_range, random_state = 77)
Bayesian.maximize(init_points = 10, n_iter = 20)

|   iter    |  target   | colsam... | learni... |  max_bin  | max_depth | min_ch... | min_sp... | num_le... | reg_la... | subsample | subsam... |
-------------------------------------------------------------------------------------------------------------------------------------------------
{'boosting': 'gbdt', 'objective': 'regression', 'metric': 'mape', 'is_training_metric': True, 'n_estimators': 25000, 'num_leaves': 26, 'max_depth': 12, 'min_child_samples': 20, 'reg_lambda': 0.5402464650464484, 'min_split_gain': 0.028640061741094756, 'colsample_bytree': 0.8676436127196727, 'subsample': 0.7480470351736012, 'subsample_freq': 4, 'max_bin': 64, 'learning_rate': 0.0067797603993146995, 'n_jobs': -1, 'device_type': 'gpu', 'verbose': -1}
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.0204405
[5000]	valid_0's mape: 0.0192287
[7500]	valid_0's mape: 0.0187755
[10000]	valid_0's mape: 0.0185971
[12500]	valid_0's mape: 0.0185147
[15000]	valid_0's mape: 0.01

In [25]:
print('Final result: ', Bayesian.max)

Final result:  {'target': -32.672035884506315, 'params': {'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_bin': 66.67327202110579, 'max_depth': 56.64261301450704, 'min_child_samples': 16.027499641612312, 'min_split_gain': 0.035, 'num_leaves': 56.1359852568734, 'reg_lambda': 0.99, 'subsample': 0.9, 'subsample_freq': 7.0}}


In [26]:
# prior은 정확한 관측치로부터 얻어진 것이 아니므로, 대략적인 정수를 사용해도 무방하다고 함!! 
lgbm_params1 = Bayesian.max["params"].copy()
lgbm_params1['num_leaves'] = int(lgbm_params1['num_leaves'])
lgbm_params1["max_depth"] = int(round(lgbm_params1["max_depth"]))
lgbm_params1["min_child_samples"] = int(round(lgbm_params1["min_child_samples"]))
lgbm_params1["subsample_freq"] = int(round(lgbm_params1["subsample_freq"]))
lgbm_params1["max_bin"] = int(round(lgbm_params1["max_bin"]))

lgbm_params1['n_jobs'] = -1
lgbm_params1['num_iterations'] = 25000
lgbm_params1['objective'] = 'regression'
lgbm_params1['metric'] = 'mape'
lgbm_params1['device_type'] = 'gpu'
lgbm_params1['verbose'] = -1
lgbm_params1['is_training_metric'] = True

In [27]:
lgbm_mape1, lgbm_pred1, lgbm_shap1 = SHAP.lgbm_SHAP(X, y2, lgbm_params1, version='0921-lgbm-bayes', cv_splits=5, scaling='MinMax', epoch=25000)

Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.018593
[5000]	valid_0's mape: 0.0182707
[7500]	valid_0's mape: 0.0182388
Early stopping, best iteration is:
[8372]	valid_0's mape: 0.0182322
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.0183726
[5000]	valid_0's mape: 0.0178995
[7500]	valid_0's mape: 0.0178412
Early stopping, best iteration is:
[8684]	valid_0's mape: 0.0178335
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.0190457
[5000]	valid_0's mape: 0.0185162
[7500]	valid_0's mape: 0.0184409
Early stopping, best iteration is:
[7890]	valid_0's mape: 0.0184394
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.0187307
[5000]	valid_0's mape: 0.0182601
[7500]	valid_0's mape: 0.0182211
Early stopping, best iteration is:
[9290]	valid_0's mape: 0.0182113
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 

In [28]:
lgbm_mape1

{'final_mape': [32.62492904340099],
 'test_mape': [33.645961897877584,
  33.40620902790818,
  33.67059906703775,
  33.00995629138737,
  33.796991191421284],
 'val_mape': [33.527382781649436,
  32.38845916283768,
  34.420364670977754,
  32.54386359404361,
  34.457433166350434]}

### lgbm_params3

In [13]:
# 20 
lgbm_params3 = {'boosting': 'gbdt', 'objective': 'regression', 'metric': 'mape', 'is_training_metric': True, 'n_estimators': 25000, 
                'num_leaves': 45, 'max_depth': 50, 'min_child_samples': 23, 'reg_lambda': 0.99, 
                'min_split_gain': 0.035, 'colsample_bytree': 0.5, 'subsample': 0.9, 'subsample_freq': 7, 'max_bin': 49, 
                'learning_rate': 0.01, 'n_jobs': -1, 'verbose': -1, 'device_type' : 'gpu'}

In [16]:
lgbm_mape3, lgbm_pred3, lgbm_shap3 = SHAP.lgbm_SHAP(X, y2, lgbm_params3, version='0921-lgbm-bayes2', cv_splits=5, scaling='MinMax', epoch=25000)

Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.0188804
[5000]	valid_0's mape: 0.0183948
[7500]	valid_0's mape: 0.018347
Early stopping, best iteration is:
[7876]	valid_0's mape: 0.018342
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.0187629
[5000]	valid_0's mape: 0.0180518
[7500]	valid_0's mape: 0.0179232
Early stopping, best iteration is:
[8660]	valid_0's mape: 0.017906
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.0194006
[5000]	valid_0's mape: 0.0186377
[7500]	valid_0's mape: 0.0184963
[10000]	valid_0's mape: 0.018457
[12500]	valid_0's mape: 0.0184438
Early stopping, best iteration is:
[12166]	valid_0's mape: 0.0184425
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.019099
[5000]	valid_0's mape: 0.0184143
[7500]	valid_0's mape: 0.018304
[10000]	valid_0's mape: 0.0182808
Early stopping, best iteration is:
[9900]	valid_0's map

In [17]:
lgbm_mape3

{'final_mape': [32.737709415296436],
 'test_mape': [33.76413752080975,
  33.667856935918444,
  33.69628657064437,
  33.215017577659474,
  33.637186255419216],
 'val_mape': [33.82646189405721,
  32.45880673687231,
  34.36072919998117,
  32.70772162239674,
  34.54003896363635]}

In [18]:
# 10 
lgbm_params4 = {'boosting': 'gbdt', 'objective': 'regression', 'metric': 'mape', 'is_training_metric': True, 'n_estimators': 25000, 
                'num_leaves': 50, 'max_depth': 16, 'min_child_samples': 21, 'reg_lambda': 0.5987452689457233, 
                'min_split_gain': 0.02219129196607859, 'colsample_bytree': 0.6124602644225087, 'subsample': 0.8818479085778064, 
                'subsample_freq': 7, 'max_bin': 70, 'learning_rate': 0.00948972736956212, 'n_jobs': -1, 'verbose': -1, 'device_type' : 'gpu'}

In [19]:
%%time
lgbm_mape4, lgbm_pred4, lgbm_shap4 = SHAP.lgbm_SHAP(X, y2, lgbm_params4, version='0921-lgbm-bayes3', cv_splits=5, scaling='MinMax', epoch=25000)

Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.0187697
[5000]	valid_0's mape: 0.018335
Early stopping, best iteration is:
[6323]	valid_0's mape: 0.0182978
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.018566
[5000]	valid_0's mape: 0.0179433
Early stopping, best iteration is:
[6517]	valid_0's mape: 0.0178829
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.0192832
[5000]	valid_0's mape: 0.0186083
[7500]	valid_0's mape: 0.0184869
Early stopping, best iteration is:
[8861]	valid_0's mape: 0.0184778
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.0189876
[5000]	valid_0's mape: 0.0184102
[7500]	valid_0's mape: 0.0183615
Early stopping, best iteration is:
[7237]	valid_0's mape: 0.0183551
Training until validation scores don't improve for 500 rounds
[2500]	valid_0's mape: 0.0189996
[5000]	valid_0's mape: 0.018531
Early stopping, best iter

In [20]:
lgbm_mape4

{'final_mape': [32.76307885844093],
 'test_mape': [33.82655631027563,
  33.55880658611452,
  33.84581804185924,
  33.20165889325133,
  33.89699116856155],
 'val_mape': [33.68555869779832,
  32.38158610479228,
  34.39264808682316,
  32.81663524053947,
  34.677726232845565]}